<a href="https://colab.research.google.com/github/sledhead/simple_python/blob/main/webpage_scrapv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
#Will display upload box to user to allow a file to upload and encrypted.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xmltodict

In [3]:
import xmltodict

In [4]:
import time

Section below will gather xml file for each year of the senate vote....
2017 to current

In [5]:
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_2.xml

!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_116_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_116_2.xml

!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_117_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_117_2.xml

!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_118_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_118_2.xml

--2024-05-23 06:01:09--  https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_1.xml
Resolving www.senate.gov (www.senate.gov)... 104.96.219.104, 2600:1408:20:c89::1e3, 2600:1408:20:ca1::1e3
Connecting to www.senate.gov (www.senate.gov)|104.96.219.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘vote_menu_115_1.xml’

vote_menu_115_1.xml     [ <=>                ] 149.40K  --.-KB/s    in 0.005s  

2024-05-23 06:01:09 (30.3 MB/s) - ‘vote_menu_115_1.xml’ saved [152990]

--2024-05-23 06:01:09--  https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_2.xml
Resolving www.senate.gov (www.senate.gov)... 104.96.219.104, 2600:1408:20:c89::1e3, 2600:1408:20:ca1::1e3
Connecting to www.senate.gov (www.senate.gov)|104.96.219.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘vote_menu_115_2.xml’

vote_menu_115_2.xml     [ <=>                ] 129.

Next Section below will open xml and then use this file to build the list of file links for gather senator votes

In [6]:

full_http_link_file_path = []
vote_menu_file_lst = ['vote_menu_115_1.xml','vote_menu_115_2.xml','vote_menu_116_1.xml','vote_menu_116_2.xml','vote_menu_117_1.xml','vote_menu_117_2.xml','vote_menu_118_1.xml','vote_menu_118_2.xml']

for each_year_call_lst in vote_menu_file_lst:

  xml_file = open(each_year_call_lst, 'r')
  xml_data_str = xml_file.read()

  xml_to_dict_obj = xmltodict.parse(xml_data_str)

  congress_num = xml_to_dict_obj['vote_summary']['congress']
  congress_sec_num = xml_to_dict_obj['vote_summary']['session']

  congress_lst_vote = xml_to_dict_obj['vote_summary']['votes']['vote']



  base_link_path = 'https://www.senate.gov/legislative/LIS/roll_call_votes/'
  full_base_link_path = base_link_path + 'vote' + str(congress_num) + str(congress_sec_num) + '/vote_' + str(congress_num) + '_' + str(congress_sec_num) + '_'
  print(full_base_link_path)

  for each_vote_section in congress_lst_vote:
    single_vote_num = each_vote_section['vote_number']
    #build full link path for getting xml data....
    #https://www.senate.gov/legislative/LIS/roll_call_votes/vote1172/vote_117_2_00420.htm

    single_file_link_path = full_base_link_path + str(single_vote_num) + '.xml'
    full_http_link_file_path.append(single_file_link_path)


https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1152/vote_115_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1161/vote_116_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1162/vote_116_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1172/vote_117_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1181/vote_118_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1182/vote_118_2_


# Test Section


In [7]:
print(len(full_http_link_file_path))

2800


In [ ]:
for single_file_path in full_http_link_file_path:
  print(single_file_path)



# Build Senator Dict

Section below will loop through list of xml files and build a dict of all the votes within the time period.  Just selected senators will be recorded.

In [8]:
def GetVoteLayout(senator_vote_list):
  #Generate additional information about the over flow of the vote.
  #Determine the party majority of both yeas and nays.
  #{'yea_major':R or D, yea_R: cnt, yea_D: cnt, yea_I:cnt, nay_major: R or D, nay_R: cnt, nay_D: cnt, nay_I:cnt}
  vote_info = {}
  yea_D_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'D' and single_senator['vote_cast'] == 'Yea') == True]
  #print(len(yea_D_lst))

  yea_R_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'R' and single_senator['vote_cast'] == 'Yea') == True]
  #print(len(yea_R_lst))

  yea_I_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'I' and single_senator['vote_cast'] == 'Yea') == True]
  #print(len(yea_I_lst))

  #calulate which party has major....
  if(len(yea_D_lst) > len(yea_R_lst)):
    vote_info['yea_major'] = 'D'
  elif(len(yea_D_lst) < len(yea_R_lst)):
    vote_info['yea_major'] = 'R'
  elif(len(yea_D_lst) == len(yea_R_lst)):
    vote_info['yea_major'] = 'E'

  vote_info['yea_D'] = len(yea_D_lst)
  vote_info['yea_R'] = len(yea_R_lst)
  vote_info['yea_I'] = len(yea_I_lst)


  #print('.........................................................')

  nays_D_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'D' and single_senator['vote_cast'] == 'Nay') == True]
  #print(len(nays_D_lst))

  nays_R_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'R' and single_senator['vote_cast'] == 'Nay') == True]
  #print(len(nays_R_lst))

  nays_I_lst = [single_senator for single_senator in senator_vote_list if( single_senator['party'] == 'I' and single_senator['vote_cast'] == 'Nay') == True]
  #print(len(nays_I_lst))

  #calulate which party has major....
  if(len(nays_D_lst) > len(nays_R_lst)):
    vote_info['nay_major'] = 'D'
  elif(len(nays_D_lst) < len(nays_R_lst)):
    vote_info['nay_major'] = 'R'
  elif(len(nays_D_lst) == len(nays_R_lst)):
    vote_info['nay_major'] = 'E'

  vote_info['nay_D'] = len(nays_D_lst)
  vote_info['nay_R'] = len(nays_R_lst)
  vote_info['nay_I'] = len(nays_I_lst)

  return vote_info

In [9]:
missing_links = []
senator_history_lst = []

#full_http_link_file_path = ['https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00271.xml', 'https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00262.xml']
select_only_senators = ['Daines', 'Tester', 'Cruz', 'Schumer','Warren','Cotton','Sanders', 'Menendez','Paul','Hirono']

for each_single_file_link in full_http_link_file_path:
  #gather the data from each single file link path
  r = requests.get(each_single_file_link)
  #print(r.status_code)
  #print(r.text)

  #make sure we got a file....
  if(r.status_code == 200):
    vote_call_details = xmltodict.parse(r.text)

    congress_id_num = vote_call_details['roll_call_vote']['congress']
    congress_session_num = vote_call_details['roll_call_vote']['session']
    vote_call_num = vote_call_details['roll_call_vote']['vote_number']
    vote_call_issue = vote_call_details['roll_call_vote']['vote_title']

    #setup loop for what each senator voted...
    senator_member_lst = vote_call_details['roll_call_vote']['members']['member']
    select_senator_vote_dict = {dict['last_name']:dict['vote_cast'] for dict in senator_member_lst if (dict['last_name'] in select_only_senators) == True }
    print(f'Currently working on: Congress: {congress_id_num}, session: {congress_session_num}, vote call: {vote_call_num}')

    new_row_dict = {}
    new_row_dict['congress'] = congress_id_num
    new_row_dict['session'] = congress_session_num
    new_row_dict['vote_number'] = vote_call_num
    new_row_dict['vote_title'] = vote_call_issue

    #add section to keep track of yea and nay counts.
    new_row_dict['yea_cnt'] = vote_call_details['roll_call_vote']['count']['yeas']
    new_row_dict['nays_cnt'] = vote_call_details['roll_call_vote']['count']['nays']
    new_row_dict['vote_deep_dive'] = GetVoteLayout(senator_member_lst)

    for each_sen_record in select_only_senators:
      #print(each_sen_mem['last_name'])

      new_row_dict[each_sen_record] = select_senator_vote_dict[each_sen_record]

      #add to list for later examation
      #after building list, will add to dataframe.

    senator_history_lst.append(new_row_dict)

  else:
    #something went wrong...
    #keep track of this problem
    missing_links.append(full_file_link_path)

  #before moving to the next record, pause for some time....
  time.sleep(0.25)

Currently working on: Congress: 115, session: 1, vote call: 325
Currently working on: Congress: 115, session: 1, vote call: 324
Currently working on: Congress: 115, session: 1, vote call: 323
Currently working on: Congress: 115, session: 1, vote call: 322
Currently working on: Congress: 115, session: 1, vote call: 321
Currently working on: Congress: 115, session: 1, vote call: 320
Currently working on: Congress: 115, session: 1, vote call: 319
Currently working on: Congress: 115, session: 1, vote call: 318
Currently working on: Congress: 115, session: 1, vote call: 317
Currently working on: Congress: 115, session: 1, vote call: 316
Currently working on: Congress: 115, session: 1, vote call: 315
Currently working on: Congress: 115, session: 1, vote call: 314
Currently working on: Congress: 115, session: 1, vote call: 313
Currently working on: Congress: 115, session: 1, vote call: 312
Currently working on: Congress: 115, session: 1, vote call: 311
Currently working on: Congress: 115, ses

# Test Section

In [10]:
#will use this section to test the above code.

select_only_senators = ['Daines', 'Tester', 'Cruz', 'Schumer', 'Baldwin', 'Bennet']

test_list = [{'member_full': 'Baldwin (D-WI)', 'last_name': 'Baldwin', 'first_name': 'Tammy', 'party': 'D', 'state': 'WI', 'vote_cast': 'Yea', 'lis_member_id': 'S354'}, {'member_full': 'Barrasso (R-WY)', 'last_name': 'Barrasso', 'first_name': 'John', 'party': 'R', 'state': 'WY', 'vote_cast': 'Not Voting', 'lis_member_id': 'S317'}, {'member_full': 'Bennet (D-CO)', 'last_name': 'Bennet', 'first_name': 'Michael', 'party': 'D', 'state': 'CO', 'vote_cast': 'Yea', 'lis_member_id': 'S330'}]
print(len(test_list))

select_senator_vote_dict = {dict['last_name']:dict['vote_cast'] for dict in test_list if (dict['last_name'] in select_only_senators) == True }

print(select_senator_vote_dict)


3
{'Baldwin': 'Yea', 'Bennet': 'Yea'}


In [ ]:
print(missing_links)

for each_vote_cnt in senator_history_lst:

  print(each_vote_cnt)

[]
{'congress': '115', 'session': '1', 'vote_number': '325', 'vote_title': 'Motion to Concur in the House Amendment to the Senate Amendment to H.R. 1370', 'yea_cnt': '66', 'nays_cnt': '32', 'Daines': 'Yea', 'Tester': 'Yea', 'Cruz': 'Yea', 'Schumer': 'Nay', 'Warren': 'Nay', 'Cotton': 'Yea', 'Sanders': 'Nay', 'Menendez': 'Nay', 'Paul': 'Nay', 'Hirono': 'Nay'}
{'congress': '115', 'session': '1', 'vote_number': '324', 'vote_title': 'Motion to Waive All Applicable Budgetary Discipline Re: The Motion to Concur in the House Amendment to the Senate Amendment to H.R. 1370', 'yea_cnt': '91', 'nays_cnt': '8', 'Daines': 'Yea', 'Tester': 'Yea', 'Cruz': 'Nay', 'Schumer': 'Yea', 'Warren': 'Yea', 'Cotton': 'Yea', 'Sanders': 'Yea', 'Menendez': 'Yea', 'Paul': 'Nay', 'Hirono': 'Yea'}
{'congress': '115', 'session': '1', 'vote_number': '323', 'vote_title': 'Motion to Recede from the Senate Amendment to H.R. 1 and Concur with Further Amendment', 'yea_cnt': '51', 'nays_cnt': '48', 'Daines': 'Yea', 'Tester': 

In [11]:
print(missing_links)
print(len(senator_history_lst))
print(senator_history_lst[155])

[]
2800
{'congress': '115', 'session': '1', 'vote_number': '170', 'vote_title': 'Donnelly Motion to Commit H.R. 1628 with Instructions', 'yea_cnt': '48', 'nays_cnt': '52', 'vote_deep_dive': {'yea_major': 'D', 'yea_D': 46, 'yea_R': 0, 'yea_I': 2, 'nay_major': 'R', 'nay_D': 0, 'nay_R': 52, 'nay_I': 0}, 'Daines': 'Nay', 'Tester': 'Yea', 'Cruz': 'Nay', 'Schumer': 'Yea', 'Warren': 'Yea', 'Cotton': 'Nay', 'Sanders': 'Yea', 'Menendez': 'Yea', 'Paul': 'Nay', 'Hirono': 'Yea'}


In [12]:
print(vote_call_details['roll_call_vote'].keys())
print(vote_call_details['roll_call_vote']['count'])
print(vote_call_details['roll_call_vote']['count']['yeas'])
print(vote_call_details['roll_call_vote']['members']['member'])

dict_keys(['congress', 'session', 'congress_year', 'vote_number', 'vote_date', 'modify_date', 'vote_question_text', 'vote_document_text', 'vote_result_text', 'question', 'vote_title', 'majority_requirement', 'vote_result', 'document', 'amendment', 'count', 'tie_breaker', 'members'])
{'yeas': '73', 'nays': '15', 'present': None, 'absent': '12'}
73
[{'member_full': 'Baldwin (D-WI)', 'last_name': 'Baldwin', 'first_name': 'Tammy', 'party': 'D', 'state': 'WI', 'vote_cast': 'Yea', 'lis_member_id': 'S354'}, {'member_full': 'Barrasso (R-WY)', 'last_name': 'Barrasso', 'first_name': 'John', 'party': 'R', 'state': 'WY', 'vote_cast': 'Not Voting', 'lis_member_id': 'S317'}, {'member_full': 'Bennet (D-CO)', 'last_name': 'Bennet', 'first_name': 'Michael', 'party': 'D', 'state': 'CO', 'vote_cast': 'Yea', 'lis_member_id': 'S330'}, {'member_full': 'Blackburn (R-TN)', 'last_name': 'Blackburn', 'first_name': 'Marsha', 'party': 'R', 'state': 'TN', 'vote_cast': 'Yea', 'lis_member_id': 'S396'}, {'member_full

In [13]:
print(len(senator_member_lst))
yea_D_lst = [single_senator for single_senator in senator_member_lst if( single_senator['party'] == 'D' and single_senator['vote_cast'] == 'Yea') == True]
print(len(yea_D_lst))

yea_R_lst = [single_senator for single_senator in senator_member_lst if( single_senator['party'] == 'R' and single_senator['vote_cast'] == 'Yea') == True]
print(len(yea_R_lst))

yea_I_lst = [single_senator for single_senator in senator_member_lst if( single_senator['party'] == 'I' and single_senator['vote_cast'] == 'Yea') == True]
print(len(yea_I_lst))

print('.........................................................')

nays_D_lst = [single_senator for single_senator in senator_member_lst if( single_senator['party'] == 'D' and single_senator['vote_cast'] == 'Nay') == True]
print(len(nays_D_lst))

nays_R_lst = [single_senator for single_senator in senator_member_lst if( single_senator['party'] == 'R' and single_senator['vote_cast'] == 'Nay') == True]
print(len(nays_R_lst))

nays_I_lst = [single_senator for single_senator in senator_member_lst if( single_senator['party'] == 'I' and single_senator['vote_cast'] == 'Nay') == True]
print(len(nays_I_lst))

100
45
25
3
.........................................................
0
15
0


In [14]:
print(len(senator_history_lst))
print(senator_history_lst[785])

2800
{'congress': '116', 'session': '1', 'vote_number': '242', 'vote_title': 'Motion to Invoke Cloture: Mark T. Pittman to be U.S. District Judge for the Northern District of Texas', 'yea_cnt': '54', 'nays_cnt': '34', 'vote_deep_dive': {'yea_major': 'R', 'yea_D': 3, 'yea_R': 51, 'yea_I': 0, 'nay_major': 'D', 'nay_D': 33, 'nay_R': 0, 'nay_I': 1}, 'Daines': 'Yea', 'Tester': 'Nay', 'Cruz': 'Yea', 'Schumer': 'Nay', 'Warren': 'Not Voting', 'Cotton': 'Yea', 'Sanders': 'Not Voting', 'Menendez': 'Nay', 'Paul': 'Yea', 'Hirono': 'Nay'}


# Senator Track Record

In [15]:
def Senator_compare_func(test_dict):
  #each single line repesent a vote for certain senators in congress
  transform_new_row = {}

  #congress year
  transform_new_row['congress'] = test_dict['congress']
  #session number
  transform_new_row['session'] = test_dict['session']
  #vote number
  transform_new_row['vote_number'] = test_dict['vote_number']

  #do simple conparison of senator
  #Daines and Tester
  if( (test_dict['Daines']) == (test_dict['Tester']) ):
    transform_new_row['Daines_Tester'] = 1

  else:
    transform_new_row['Daines_Tester'] = 0

  #Daines and Rep Group
  if( (test_dict['Daines'] == test_dict['Cruz']) and (test_dict['Daines'] == test_dict['Cotton']) ):
    transform_new_row['Daines_Rep_Group'] = 1

  else:
    transform_new_row['Daines_Rep_Group'] = 0

  #Tester and Dem Group
  if( (test_dict['Tester'] == test_dict['Schumer']) and (test_dict['Tester'] == test_dict['Menendez']) ):
    transform_new_row['Tester_Dem_Group'] = 1

  else:
    transform_new_row['Tester_Dem_Group'] = 0

  #compare Daines = Tester but not Schumer
  if( (test_dict['Daines'] == test_dict['Tester']) and (test_dict['Schumer'] != test_dict['Daines']) ):
    transform_new_row['Daines_Tester_Schemer'] = 1

  else:
    transform_new_row['Daines_Tester_Schemer'] = 0


  r_vote_pos = 'N/A'
  d_vote_pos = 'N/A'

  if( test_dict['vote_deep_dive']['yea_R'] > test_dict['vote_deep_dive']['nay_R']):
    r_vote_pos = 'Yea'
  elif( test_dict['vote_deep_dive']['yea_R'] < test_dict['vote_deep_dive']['nay_R']):
    r_vote_pos = 'Nay'
  elif( test_dict['vote_deep_dive']['yea_R'] == test_dict['vote_deep_dive']['nay_R']):
    if(test_dict['vote_deep_dive']['yea_D'] > test_dict['vote_deep_dive']['nay_D']):
      r_vote_pos = 'Nay'
    else:
      r_vote_pos = 'Yea'


  if( test_dict['vote_deep_dive']['yea_D'] > test_dict['vote_deep_dive']['nay_D']):
    d_vote_pos = 'Yea'
  elif( test_dict['vote_deep_dive']['yea_D'] < test_dict['vote_deep_dive']['nay_D']):
    d_vote_pos = 'Nay'
  elif( test_dict['vote_deep_dive']['yea_D'] == test_dict['vote_deep_dive']['nay_D']):
    if(test_dict['vote_deep_dive']['yea_R'] > test_dict['vote_deep_dive']['nay_R']):
      d_vote_pos = 'Nay'
    else:
      d_vote_pos = 'Yea'

  if( (test_dict['Daines'] == 'Yea' and r_vote_pos == 'Yea') or (test_dict['Daines'] == 'Nay' and r_vote_pos == 'Nay') ):
    transform_new_row['Daines_Vote_A_Party'] = 0
  elif( (test_dict['Daines'] == 'Yea' and r_vote_pos == 'Nay') or (test_dict['Daines'] == 'Nay' and r_vote_pos == 'Yea') ):
    transform_new_row['Daines_Vote_A_Party'] = 1
  else:
    transform_new_row['Daines_Vote_A_Party'] = 9


  if( (test_dict['Tester'] == 'Yea' and d_vote_pos == 'Yea') or (test_dict['Tester'] == 'Nay' and d_vote_pos == 'Nay') ):
    transform_new_row['Tester_Vote_A_Party'] = 0
  elif( (test_dict['Tester'] == 'Yea' and d_vote_pos == 'Nay') or (test_dict['Tester'] == 'Nay' and d_vote_pos == 'Yea') ):
    transform_new_row['Tester_Vote_A_Party'] = 1
  else:
    transform_new_row['Tester_Vote_A_Party'] = 9

  #how many times did Daines vote against the R majority....


  return transform_new_row

In [16]:
simple_senator_compare = [Senator_compare_func(dict_obj) for dict_obj in senator_history_lst]

# Test Section

In [ ]:
print(len(simple_senator_compare))

2795


In [ ]:
print(simple_senator_compare[376])
print(senator_history_lst[376])

{'congress': '115', 'session': '2', 'vote_number': '223', 'Daines_Tester': 0, 'Daines_Rep_Group': 0, 'Tester_Dem_Group': 1, 'Daines_Tester_Schemer': 0, 'Daines_Vote_A_Party': 9, 'Tester_Vote_A_Party': 0}
{'congress': '115', 'session': '2', 'vote_number': '223', 'vote_title': 'Confirmation Brett M. Kavanaugh, of Maryland, to be an Associate Justice of the Supreme Court of the United States', 'yea_cnt': '50', 'nays_cnt': '48', 'vote_deep_dive': {'yea_major': 'R', 'yea_D': 1, 'yea_R': 49, 'yea_I': 0, 'nay_major': 'D', 'nay_D': 46, 'nay_R': 0, 'nay_I': 2}, 'Daines': 'Not Voting', 'Tester': 'Nay', 'Cruz': 'Yea', 'Schumer': 'Nay', 'Warren': 'Nay', 'Cotton': 'Yea', 'Sanders': 'Nay', 'Menendez': 'Nay', 'Paul': 'Yea', 'Hirono': 'Nay'}


In [ ]:
print(simple_senator_compare[239])
how_big = [dict_obj for dict_obj in simple_senator_compare if dict_obj['Daines_Tester_Schemer'] == 1]
print(len(how_big))

how_big = [dict_obj for dict_obj in simple_senator_compare if dict_obj['Daines_Rep_Group'] == 1]
print(len(how_big))

how_big = [dict_obj for dict_obj in simple_senator_compare if dict_obj['Tester_Dem_Group'] == 1]
print(len(how_big))

how_big = [dict_obj for dict_obj in simple_senator_compare if dict_obj['Daines_Vote_A_Party'] == 1]
print(f'How many times Daines voted against the Rep Party {len(how_big)}')

how_big = [dict_obj for dict_obj in simple_senator_compare if dict_obj['Tester_Vote_A_Party'] == 1]
print(f'How many times Tester voted against the Dec Party {len(how_big)}')


{'congress': '115', 'session': '1', 'vote_number': '86', 'Daines_Tester': 0, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 1, 'Daines_Tester_Schemer': 0, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 0}
286
2233
2250
How many times Daines voted against the Rep Party 178
How many times Tester voted against the Dec Party 193


In [ ]:
for each_vote in how_big:
  print(each_vote)

{'congress': '115', 'session': '1', 'vote_number': '325', 'Daines_Tester': 1, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 0, 'Daines_Tester_Schemer': 1, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 1}
{'congress': '115', 'session': '1', 'vote_number': '319', 'Daines_Tester': 1, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 0, 'Daines_Tester_Schemer': 1, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 1}
{'congress': '115', 'session': '1', 'vote_number': '318', 'Daines_Tester': 1, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 0, 'Daines_Tester_Schemer': 1, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 1}
{'congress': '115', 'session': '1', 'vote_number': '305', 'Daines_Tester': 1, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 0, 'Daines_Tester_Schemer': 1, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 1}
{'congress': '115', 'session': '1', 'vote_number': '304', 'Daines_Tester': 1, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 0, 'Daines_Tester_Schemer': 1, 'Daines_Vote_A_Party': 0, 'Teste

In [ ]:
for st in simple_senator_compare:
  print(st)

{'congress': '115', 'session': '1', 'vote_number': '325', 'Daines_Tester': 1, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 0, 'Daines_Tester_Schemer': 1, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 1}
{'congress': '115', 'session': '1', 'vote_number': '324', 'Daines_Tester': 1, 'Daines_Rep_Group': 0, 'Tester_Dem_Group': 1, 'Daines_Tester_Schemer': 0, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 0}
{'congress': '115', 'session': '1', 'vote_number': '323', 'Daines_Tester': 0, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 1, 'Daines_Tester_Schemer': 0, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 0}
{'congress': '115', 'session': '1', 'vote_number': '322', 'Daines_Tester': 0, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 1, 'Daines_Tester_Schemer': 0, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 0}
{'congress': '115', 'session': '1', 'vote_number': '321', 'Daines_Tester': 0, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 1, 'Daines_Tester_Schemer': 0, 'Daines_Vote_A_Party': 0, 'Teste

In [ ]:
if( test_dict['Daines'] == 'Yea'):
    if( test_dict['vote_deep_dive']['yea_major'] == 'R'):
      if( test_dict['vote_deep_dive']['yea_R'] > test_dict['vote_deep_dive']['nay_R'] ):
        transform_new_row['Daines_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['yea_R'] < test_dict['vote_deep_dive']['nay_R']):
        transform_new_row['Daines_Vote_A_Party'] = 1
      else:
        transform_new_row['Daines_Vote_A_Party'] = 0

    elif( test_dict['vote_deep_dive']['yea_major'] == 'D'):
      if( test_dict['vote_deep_dive']['yea_R'] > test_dict['vote_deep_dive']['nay_R'] ):
        transform_new_row['Daines_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['yea_R'] < test_dict['vote_deep_dive']['nay_R']):
        transform_new_row['Daines_Vote_A_Party'] = 1
      else:
        transform_new_row['Daines_Vote_A_Party'] = 0

    elif( test_dict['vote_deep_dive']['yea_major'] == 'E'):
      if( test_dict['vote_deep_dive']['yea_R'] > test_dict['vote_deep_dive']['nay_R'] ):
        transform_new_row['Daines_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['yea_R'] < test_dict['vote_deep_dive']['nay_R']):
        transform_new_row['Daines_Vote_A_Party'] = 1

  elif( test_dict['Daines'] == 'Nay'):
    if( test_dict['vote_deep_dive']['nay_major'] == 'R'):
      if( test_dict['vote_deep_dive']['nay_R'] > test_dict['vote_deep_dive']['yea_R'] ):
        transform_new_row['Daines_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['nay_R'] < test_dict['vote_deep_dive']['yea_R']):
        transform_new_row['Daines_Vote_A_Party'] = 1
      else:
        transform_new_row['Daines_Vote_A_Party'] = 0

    elif( test_dict['vote_deep_dive']['nay_major'] == 'D'):
      if( test_dict['vote_deep_dive']['nay_R'] > test_dict['vote_deep_dive']['yea_R'] ):
        transform_new_row['Daines_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['nay_R'] < test_dict['vote_deep_dive']['yea_R']):
        transform_new_row['Daines_Vote_A_Party'] = 1
      else:
        transform_new_row['Daines_Vote_A_Party'] = 0

    elif( test_dict['vote_deep_dive']['nay_major'] == 'E'):
      if( test_dict['vote_deep_dive']['nay_R'] > test_dict['vote_deep_dive']['yea_R'] ):
        transform_new_row['Daines_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['nay_R'] < test_dict['vote_deep_dive']['yea_R']):
        transform_new_row['Daines_Vote_A_Party'] = 1

  else:
    transform_new_row['Daines_Vote_A_Party'] = -1



  #how many times did Tester vote against the D majority....
  if( test_dict['Tester'] == 'Yea'):
    if( test_dict['vote_deep_dive']['yea_major'] == 'R'):
      if( test_dict['vote_deep_dive']['yea_D'] > test_dict['vote_deep_dive']['nay_D'] ):
        transform_new_row['Tester_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['yea_D'] < test_dict['vote_deep_dive']['nay_D']):
        transform_new_row['Tester_Vote_A_Party'] = 1
      else:
        transform_new_row['Tester_Vote_A_Party'] = 0

    elif( test_dict['vote_deep_dive']['yea_major'] == 'D'):
      if( test_dict['vote_deep_dive']['yea_D'] > test_dict['vote_deep_dive']['nay_D'] ):
        transform_new_row['Tester_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['yea_D'] < test_dict['vote_deep_dive']['nay_D']):
        transform_new_row['Tester_Vote_A_Party'] = 1
      else:
        transform_new_row['Tester_Vote_A_Party'] = 0

    elif( test_dict['vote_deep_dive']['yea_major'] == 'E'):
      if( test_dict['vote_deep_dive']['yea_D'] > test_dict['vote_deep_dive']['nay_D'] ):
        transform_new_row['Tester_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['yea_D'] < test_dict['vote_deep_dive']['nay_D']):
        transform_new_row['Tester_Vote_A_Party'] = 1

  elif( test_dict['Tester'] == 'Nay'):
    if( test_dict['vote_deep_dive']['nay_major'] == 'R'):
      if( test_dict['vote_deep_dive']['nay_D'] > test_dict['vote_deep_dive']['yea_D'] ):
        transform_new_row['Tester_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['nay_D'] < test_dict['vote_deep_dive']['yea_D']):
        transform_new_row['Tester_Vote_A_Party'] = 1
      else:
        transform_new_row['Tester_Vote_A_Party'] = 0

    elif( test_dict['vote_deep_dive']['nay_major'] == 'D'):
      if( test_dict['vote_deep_dive']['nay_D'] > test_dict['vote_deep_dive']['yea_D'] ):
        transform_new_row['Tester_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['nay_D'] < test_dict['vote_deep_dive']['yea_D']):
        transform_new_row['Tester_Vote_A_Party'] = 1
      else:
        transform_new_row['Tester_Vote_A_Party'] = 0

    elif( test_dict['vote_deep_dive']['nay_major'] == 'E'):
      if( test_dict['vote_deep_dive']['nay_D'] > test_dict['vote_deep_dive']['yea_D'] ):
        transform_new_row['Tester_Vote_A_Party'] = 0
      elif(test_dict['vote_deep_dive']['nay_D'] < test_dict['vote_deep_dive']['yea_D']):
        transform_new_row['Tester_Vote_A_Party'] = 1

  else:
    transform_new_row['Tester_Vote_A_Party'] = -1

In [ ]:
print(senator_history_lst[376])

{'congress': '115', 'session': '2', 'vote_number': '223', 'vote_title': 'Confirmation Brett M. Kavanaugh, of Maryland, to be an Associate Justice of the Supreme Court of the United States', 'yea_cnt': '50', 'nays_cnt': '48', 'vote_deep_dive': {'yea_major': 'R', 'yea_D': 1, 'yea_R': 49, 'yea_I': 0, 'nay_major': 'D', 'nay_D': 46, 'nay_R': 0, 'nay_I': 2}, 'Daines': 'Not Voting', 'Tester': 'Nay', 'Cruz': 'Yea', 'Schumer': 'Nay', 'Warren': 'Nay', 'Cotton': 'Yea', 'Sanders': 'Nay', 'Menendez': 'Nay', 'Paul': 'Yea', 'Hirono': 'Nay'}


In [ ]:
senator_range_compare = [dict_obj for dict_obj in senator_history_lst if(abs(int(dict_obj['yea_cnt']) - int(dict_obj['nays_cnt'] )) < 10) == True ]
print(len(senator_range_compare))

1058


In [ ]:
range_senator_compare = [Senator_compare_func(dict_obj) for dict_obj in senator_range_compare]

In [ ]:
how_big = [dict_obj for dict_obj in range_senator_compare if dict_obj['Daines_Tester_Schemer'] == 1]
print(len(how_big))

how_big = [dict_obj for dict_obj in range_senator_compare if dict_obj['Daines_Rep_Group'] == 1]
print(len(how_big))

how_big = [dict_obj for dict_obj in range_senator_compare if dict_obj['Tester_Dem_Group'] == 1]
print(len(how_big))

how_big = [dict_obj for dict_obj in range_senator_compare if dict_obj['Daines_Vote_A_Party'] == 1]
print(f'How many times Daines voted against the Rep Party {len(how_big)}')

how_big = [dict_obj for dict_obj in range_senator_compare if dict_obj['Tester_Vote_A_Party'] == 1]
print(f'How many times Tester voted against the Dec Party {len(how_big)}')

28
974
949
How many times Daines voted against the Rep Party 7
How many times Tester voted against the Dec Party 27


In [ ]:
for each_sp in how_big:
  print(each_sp)

{'congress': '115', 'session': '1', 'vote_number': '20', 'Daines_Tester': 1, 'Daines_Rep_Group': 0, 'Tester_Dem_Group': 0, 'Daines_Tester_Schemer': 1, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 1}
{'congress': '115', 'session': '2', 'vote_number': '174', 'Daines_Tester': 1, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 0, 'Daines_Tester_Schemer': 1, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 1}
{'congress': '115', 'session': '2', 'vote_number': '172', 'Daines_Tester': 1, 'Daines_Rep_Group': 1, 'Tester_Dem_Group': 0, 'Daines_Tester_Schemer': 1, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 1}
{'congress': '115', 'session': '2', 'vote_number': '111', 'Daines_Tester': 1, 'Daines_Rep_Group': 0, 'Tester_Dem_Group': 0, 'Daines_Tester_Schemer': 1, 'Daines_Vote_A_Party': 0, 'Tester_Vote_A_Party': 1}
{'congress': '115', 'session': '2', 'vote_number': '110', 'Daines_Tester': 1, 'Daines_Rep_Group': 0, 'Tester_Dem_Group': 0, 'Daines_Tester_Schemer': 1, 'Daines_Vote_A_Party': 0, 'Tester

# Party Balance

In [17]:
def Compute_Party_Balance(test_dict):
  #each single line repesent a vote for certain senators in congress
  transform_new_row = {}

  #congress year
  transform_new_row['congress'] = test_dict['congress']
  #session number
  transform_new_row['session'] = test_dict['session']
  #vote number
  transform_new_row['vote_number'] = test_dict['vote_number']

  #determine the balance of each party... balance vote or one sided...
  transform_new_row['R_vote_bal'] = abs(int(test_dict['vote_deep_dive']['yea_R']) - int(test_dict['vote_deep_dive']['nay_R']))
  transform_new_row['R_vote_tot'] = int(test_dict['vote_deep_dive']['yea_R']) + int(test_dict['vote_deep_dive']['nay_R'])

  transform_new_row['D_vote_bal'] = abs(int(test_dict['vote_deep_dive']['yea_D']) - int(test_dict['vote_deep_dive']['nay_D']))
  transform_new_row['D_vote_tot'] = int(test_dict['vote_deep_dive']['yea_D']) + int(test_dict['vote_deep_dive']['nay_D'])




  return transform_new_row

In [18]:
senate_party_balance_lst = [Compute_Party_Balance(dict_obj) for dict_obj in senator_history_lst]

# Test Section